<details>
<summary>Table of Contents</summary>

- [1. No RevIN](#1-no-revin-instanse-normalization)
- [2. No channel-independence (Channel-Mixing)](#2-no-channel-independence-channel-mixing)
- [3. No channel-independence (Channel-Mixing) & No RevIN](#3-no-channel-independence-channel-mixing-and-no-revin)
- [3. No Patching](#4-no-patching)


</details>

Ablation study on PatchTST components.

In [1]:
import os
import pandas as pd
import subprocess
import shutil
import time
from utils.helper import extract_metrics_from_output, convert_results_into_df, running_time

# 1. No RevIN (Instanse Normalization)

In [3]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

log_dir = f"logs/patchtst/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [4]:
cuda_device = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = [5, 5, 3, 3, 3]
seq_len = 336
model = "PatchTST"
loss = "MAE"
itr=2

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_no_revin.log"

# Parameters for tuning,but default
lr = 0.0001
n_heads = 16
e_layers = 3
d_model = 128
d_ff = 256
dropout = 0.2
batch_size = 128

# List to store the results
patchtst_results = []

In [4]:
start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --revin 0 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=0, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0

In [5]:
shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['-RevIN'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_no_revin.csv'))
patchtst_df.round(4)

Model             -RevIN                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0218  0.1477  0.0918
        96        0.0405  0.2013  0.1327
        168       0.0437  0.2092  0.1401
ES      24        0.0128  0.1130  0.0731
        96        0.0249  0.1579  0.1055
        168       0.0311  0.1764  0.1167
FR      24        0.0107  0.1036  0.0600
        96        0.0196  0.1399  0.0823
        168       0.0243  0.1558  0.0904
GB      24        0.0261  0.1616  0.1044
        96        0.0460  0.2144  0.1477
        168       0.0494  0.2222  0.1537
IT      24        0.0107  0.1035  0.0611
        96        0.0187  0.1369  0.0835
        168       0.0202  0.1423  0.0877

# 2. No channel independence (Channel-Mixing)

It is a channel mixing model, and therefore it needs more dimension of embeddings to capture complex patterns between features. 

Therefore, it is not fair to keep same d_model and d_ff as in channel mixing. In this regard, we scale them based on number of input features.

In other words, for DE data with 5 columns, d_model = 128 x 5, and d_ff = 256 x 5.

For ES: d_model = 128 x 3 and d_ff = 256 x 3, etc. It is adjusted automatically in code.


In [6]:
# List to store the results
patchtst_results = []

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_channel_mixing.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)
        
        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --channel_mixing 1 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=1, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0

FileNotFoundError: [Errno 2] No such file or directory: './checkpoints'

In [19]:
shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['CM'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_channel_mixing.csv'))
patchtst_df.round(4)

Model                 CM                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0209  0.1447  0.0877
        96        0.0376  0.1940  0.1282
        168       0.0393  0.1983  0.1340
ES      24        0.0098  0.0989  0.0593
        96        0.0186  0.1365  0.0865
        168       0.0209  0.1445  0.0922
FR      24        0.0100  0.0998  0.0548
        96        0.0192  0.1384  0.0804
        168       0.0215  0.1465  0.0872
GB      24        0.0249  0.1577  0.0995
        96        0.0417  0.2041  0.1395
        168       0.0447  0.2115  0.1468
IT      24        0.0101  0.1004  0.0567
        96        0.0188  0.1372  0.0808
        168       0.0209  0.1447  0.0870

# 3. No channel independence (channel-mixing) and no ReVIN

In [27]:
# List to store the results
patchtst_results = []

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_channel_mixing_no_revin.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --channel_mixing 1 \
              --revin 0 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=1, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=0, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0

In [28]:
shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['- RevIn & CM'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_channel_mixing_no_revin.csv'))
patchtst_df.round(4)

Model            - RevIn & CM                
Metrics                   MSE    RMSE     MAE
Country Pred_len                             
DE      24             0.0218  0.1477  0.0918
        96             0.0405  0.2013  0.1327
        168            0.0437  0.2092  0.1401
ES      24             0.0128  0.1130  0.0731
        96             0.0249  0.1579  0.1055
        168            0.0311  0.1764  0.1167
FR      24             0.0107  0.1036  0.0600
        96             0.0196  0.1399  0.0823
        168            0.0243  0.1558  0.0904
GB      24             0.0261  0.1616  0.1044
        96             0.0460  0.2144  0.1477
        168            0.0494  0.2222  0.1537
IT      24             0.0107  0.1035  0.0611
        96             0.0187  0.1369  0.0835
        168            0.0202  0.1423  0.0877

# 4. No patching

It runs more than 24 hours on 48GB GPU (1 country around 5-6 hours). Therefore I run it with portions. You can find full results in logs.

In [4]:
# Initialize empty list
patchtst_results = []

patch_len = 1
stride = 1

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_no_patching.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len {patch_len} \
              --stride {stride} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=1, stride=1, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=128, patience=5, learning_rate=0.0001

KeyboardInterrupt: 

In [5]:
shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['- P'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_no_patching.csv'))
patchtst_df.round(4)

Model                - P                
Metrics              MSE    RMSE     MAE
Country Pred_len                        
DE      24        0.0217  0.1472  0.0909
        96        0.0380  0.1949  0.1290
        168       0.0402  0.2004  0.1349
ES      24        0.0107  0.1034  0.0632
        96        0.0193  0.1390  0.0894
        168       0.0217  0.1472  0.0960
FR      24        0.0108  0.1040  0.0585
        96        0.0205  0.1432  0.0837
        168       0.0216  0.1471  0.0878
GB      24        0.0264  0.1626  0.1037
        96        0.0421  0.2052  0.1391
        168       0.0444  0.2106  0.1445
IT      24        0.0109  0.1044  0.0594
        96        0.0192  0.1387  0.0825
        168       0.0202  0.1422  0.0865

# 5. TS Decomposition

In [ ]:
# List to store the results
patchtst_results = []

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_decomposition.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --decomposition 1 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=1, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0

In [9]:
shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['Decomposition'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_decomposition.csv'))
patchtst_df.round(4)

Model            Decomposition                
Metrics                    MSE    RMSE     MAE
Country Pred_len                              
DE      24              0.0216  0.1468  0.0904
        96              0.0383  0.1956  0.1297
        168             0.0416  0.2039  0.1374
ES      24              0.0113  0.1060  0.0663
        96              0.0217  0.1470  0.0959
        168             0.0262  0.1610  0.1060
FR      24              0.0103  0.1013  0.0567
        96              0.0194  0.1394  0.0815
        168             0.0224  0.1497  0.0889
GB      24              0.0261  0.1615  0.1035
        96              0.0445  0.2109  0.1449
        168             0.0478  0.2187  0.1512
IT      24              0.0103  0.1017  0.0583
        96              0.0186  0.1364  0.0814
        168             0.0203  0.1425  0.0868

# 6. TS Decomposition + No RevIN

In [12]:
patchtst_results = []
# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}_decomposition_no_revin.log"

start = time.time()

# Log file
with open(log_file_path, "w") as log_file:

    for i, country in enumerate(countries):
        
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:

            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_{country}"
            dataset = f"{country}_data.csv"
            
            # Arguments for the command
            command = f"""
            python {script_path} \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers {e_layers} \
              --factor 1 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --train_epochs 100 \
              --patience 10 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --overlapping_windows \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc {loss} \
              --decomposition 1 \
              --revin 0 \
              --itr {itr} --batch_size {batch_size} --learning_rate "{lr}"
            """

            # Log the country and prediction length
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            process.wait()  # Wait for the process to finish
            shutil.rmtree('./checkpoints' )  # delete checkpoint files

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)

            # Log the extracted metrics and save them
            for iteration, scaled_metrics in enumerate(iteration_metrics, start=1):

                patchtst_results.append({
                    'Country': country,
                    'Pred_len': pred_len,
                    'Iteration': iteration,
                    'MSE': scaled_metrics[0],
                    'RMSE': scaled_metrics[1],
                    'MAE': scaled_metrics[2],
                    })
                
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = "Intermediate time for {} and pred_len {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, pred_len, hours_int, mins_int, secs_int)
            log_file.write(statement_3)
            print(statement_3)

        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = "Intermediate time for {}: {:0>2}h:{:0>2}m:{:05.2f}s".format(country, hours_c, mins_c, secs_c)
        log_file.write(statement_4)
        print(statement_4)

    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = "Total time: {:0>2}h:{:0>2}m:{:05.2f}s".format(hours, mins, secs)
    log_file.write(statement_5)
    print(statement_5)


=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_336_24_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=False, loss_fnc='MAE', fc_dropout=0.2, head_dropout=0.0, patch_len=32, stride=16, padding_patch='end', revin=0, affine=0, subtract_last=0, decomposition=1, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=7, c_out=5, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=100, batch_size=128, patience=10, learning_rate=0.

In [13]:
shutil.rmtree("results_transformers") # we do not need this directory and results anymore. If you need - comment this line

path = 'results/patchtst'
patchtst_df = convert_results_into_df(patchtst_results, if_loss_fnc=False)

if not os.path.exists(path):
    os.makedirs(path)

# Final DF
patchtst_df.columns = pd.MultiIndex.from_product([['-RevIN + Decomposition '], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])
patchtst_df.to_csv(os.path.join(path, 'patchtst_decomposition_no_revin.csv'))
patchtst_df.round(4)

Model            -RevIN + Decomposition                 
Metrics                              MSE    RMSE     MAE
Country Pred_len                                        
DE      24                        0.0217  0.1474  0.0912
        96                        0.0407  0.2015  0.1325
        168                       0.0425  0.2060  0.1390
ES      24                        0.0158  0.1228  0.0743
        96                        0.0291  0.1677  0.1059
        168                       0.0314  0.1747  0.1126
FR      24                        0.0111  0.1053  0.0598
        96                        0.0215  0.1463  0.0856
        168                       0.0243  0.1554  0.0925
GB      24                        0.0266  0.1631  0.1045
        96                        0.0489  0.2210  0.1490
        168                       0.0518  0.2275  0.1552
IT      24                        0.0108  0.1041  0.0611
        96                        0.0190  0.1378  0.0836
        168                       0.0206  0.1434  0.0886